# Create interactive plots

This notebook is to assist in exploring the phenology results

In [ ]:
%matplotlib inline

import sys
import warnings
import odc.geo.xr
import xarray as xr
import branca.colormap as cm
import matplotlib as mpl
from odc.geo.xr import assign_crs

sys.path.append('/g/data/os22/chad_tmp/AusEFlux/src/')
from _utils import round_coords

## Analysis Parameters


In [ ]:
lin_or_circ = 'circular'

p_average = assign_crs(xr.open_dataset(f'/g/data/os22/chad_tmp/Aus_phenology/results/mean_phenology_perpixel_{lin_or_circ}.nc'), crs='EPSG:4326')
p_trends = assign_crs(xr.open_dataset(f'/g/data/os22/chad_tmp/Aus_phenology/results/trends_phenology_perpixel_{lin_or_circ}.nc'), crs='EPSG:4326')
# p_parcorr = assign_crs(xr.open_dataset(f'/g/data/os22/chad_tmp/Aus_phenology/results/ios_analysis_perpixel_{lin_or_circ}.nc'), crs='EPSG:4326')

## Masking

In [ ]:
season_per_year = p_average['n_seasons']/p_average['n_years']
non_seasonal = xr.where((season_per_year <= 0.90),1,0)
extra_seasonal = xr.where((season_per_year >= 1.1),1,0)
seasonality_mask = (non_seasonal | extra_seasonal)

## Long-term average phenology




In [ ]:
var = 'SOS'
cmap='twilight'
vmin, vmax= 0, 365
m = p_average[var].odc.explore(
            tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
            attr = 'Esri',
            name = 'Esri Satellite',
            cmap=cmap,
            vmin=vmin,
            vmax=vmax
)

cmaps = mpl.colormaps[cmap]
colormap = cm.LinearColormap(cmaps.colors, vmin=vmin,vmax=vmax, caption=var)
m.add_child(colormap)

In [ ]:
var = 'EOS'
cmap='twilight'
vmin, vmax= 0, 365
m = p_average[var].odc.explore(
            tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
            attr = 'Esri',
            name = 'Esri Satellite',
            cmap=cmap,
            vmin=vmin,
            vmax=vmax
)

cmaps = mpl.colormaps[cmap]
colormap = cm.LinearColormap(cmaps.colors, vmin=vmin,vmax=vmax, caption=var)
m.add_child(colormap)

## Trends

In [ ]:
var = 'SOS'
cmap='RdBu_r'
vmin, vmax= -2,2

m = p_trends[var+'_slope'].odc.explore( #.where(p_trends[var+'_p_value']<=0.05)
            tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
            attr = 'Esri',
            name = 'Esri Satellite',
            cmap=cmap,
            vmin=vmin,
            vmax=vmax
)
m

In [ ]:
var = 'POS'
cmap='RdBu_r'
vmin, vmax= -2,2

m = p_trends[var+'_slope'].where(seasonality_mask!=1).odc.explore( #.where(p_trends[var+'_p_value']<=0.05)
            tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
            attr = 'Esri',
            name = 'Esri Satellite',
            cmap=cmap,
            vmin=vmin,
            vmax=vmax
)
m
# cmaps = mpl.colormaps[cmap]
# colormap = cm.LinearColormap(cmaps, vmin=vmin, vmax=vmax, caption=var)
# m.add_child(colormap)

In [ ]:
var = 'EOS'
cmap='RdBu_r'
vmin, vmax= -2,2

m = p_trends[var+'_slope'].odc.explore( #.where(p_trends[var+'_p_value']<=0.05)
            tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
            attr = 'Esri',
            name = 'Esri Satellite',
            cmap=cmap,
            vmin=vmin,
            vmax=vmax
)
m

In [ ]:
var = 'IOS'
cmap='PuOr'
vmin, vmax= -1,1

m = p_trends[var+'_slope'].where(p_trends[var+'_p_value']<=0.05).odc.explore(
            tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
            attr = 'Esri',
            name = 'Esri Satellite',
            cmap=cmap,
            vmin=vmin,
            vmax=vmax
)
m

In [ ]:
var = 'AOS'
cmap='PiYG'
vmin, vmax= -0.0025, 0.0025

m = p_trends[var+'_slope'].where(p_trends[var+'_p_value']<=0.05).odc.explore( #.where(p_trends[var+'_p_value']<=0.05)
            tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
            attr = 'Esri',
            name = 'Esri Satellite',
            cmap=cmap,
            vmin=vmin,
            vmax=vmax
)
m

In [ ]:
vmin,vmax=0.7,1.3
cmap='RdYlBu'

m = season_per_year.odc.explore(
            tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
            attr = 'Esri',
            name = 'Esri Satellite',
            cmap=cmap,
            vmin=vmin,
            vmax=vmax
)
m